<a target="_blank" href="https://colab.research.google.com/github/jgomelop/ai4eng_CusFraDect/blob/master/2%20-%20Preprocesado%20de%20datos.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [2]:
!gdown https://drive.google.com/uc?id=1H0OEYX33Qj-ggnlic0ILN3tcwIRbGATt

Downloading...
From: https://drive.google.com/uc?id=1H0OEYX33Qj-ggnlic0ILN3tcwIRbGATt
To: /content/ieee-fraud-detection.zip
100% 124M/124M [00:01<00:00, 104MB/s]


In [3]:
!mkdir data
!unzip ieee-fraud-detection.zip -d data
!rm -r sample_data/

Archive:  ieee-fraud-detection.zip
  inflating: data/sample_submission.csv  
  inflating: data/test_identity.csv  
  inflating: data/test_transaction.csv  
  inflating: data/train_identity.csv  
  inflating: data/train_transaction.csv  


In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gc

In [5]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
  """
  From https://www.kaggle.com/code/kabure/extensive-eda-and-modeling-xgb-hyperopt/notebook

  """
  numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  start_mem = df.memory_usage().sum() / 1024**2    
  for col in df.columns:
      col_type = df[col].dtypes
      if col_type in numerics:
          c_min = df[col].min()
          c_max = df[col].max()
          if str(col_type)[:3] == 'int':
              if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                  df[col] = df[col].astype(np.int8)
              elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                  df[col] = df[col].astype(np.int16)
              elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                  df[col] = df[col].astype(np.int32)
              elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                  df[col] = df[col].astype(np.int64)  
          else:
              if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                  df[col] = df[col].astype(np.float16)
              elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                  df[col] = df[col].astype(np.float32)
              else:
                  df[col] = df[col].astype(np.float64)    
  end_mem = df.memory_usage().sum() / 1024**2
  if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
  return df
  

In [6]:
train_tr = pd.read_csv("data/train_transaction.csv")
train_tr = reduce_mem_usage(train_tr)
train_tr.head()

Mem. usage decreased to 542.35 Mb (69.4% reduction)


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
train_tr.shape

(590540, 394)

In [7]:
train_id = pd.read_csv("data/train_identity.csv")
train_id = reduce_mem_usage(train_id)
train_id.head()

Mem. usage decreased to 25.86 Mb (42.7% reduction)


,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [46]:
train_id.shape

(144233, 41)

In [8]:
train = pd.merge(train_tr, train_id, on='TransactionID', how='left')
train = reduce_mem_usage(train)

Mem. usage decreased to 650.48 Mb (0.0% reduction)


In [45]:
train.shape

(590540, 178)

Vamos a eliminar las columnas que tengan más del 80% de los datos nulos

In [9]:
many_null_cols = [col for col in train.columns if train[col].isnull().sum() / train.shape[0] > 0.8]

In [10]:
many_null_cols[:10]

['dist2', 'D6', 'D7', 'D8', 'D9', 'D12', 'D13', 'D14', 'V138', 'V139']

In [11]:
one_value_cols = [col for col in train.columns if train[col].nunique() <= 1]
one_value_cols

[]

En la exploración de datos se definen estas columnas como importantes (recordar que las columnas Vxxx son 339 en total, y estaban altamente correlacionadas.)

In [12]:
v_numbers_to_keep = [1, 3, 4, 6, 8, 11, 13, 14, 17, 20, 23, 26, 27, 30, 36, 37, 40, 
                  41, 44, 47, 48, 54, 56, 59, 62, 65, 67, 68, 70, 76, 78, 80, 82, 
                  86, 88, 89, 91, 96, 98, 99, 104, 107, 108, 111, 115, 117, 120, 
                  121, 123, 124, 127, 129, 130, 136, 138, 139, 142, 147, 156, 162, 
                  165, 160, 166, 178, 176, 173, 182, 169, 171, 175, 180, 185, 
                  188, 198, 210, 209, 218, 223, 224, 226, 228, 229, 235, 240, 258, 
                  257, 253, 252, 260, 261, 264, 266, 267, 274, 277, 220, 221, 234, 
                  238, 250, 271, 294, 284, 285, 286, 291, 297, 303, 305, 307, 309, 
                  310, 320, 281, 283, 289, 296, 301, 314, 332, 325, 335, 338]

In [13]:
v_cols_to_drop = ["V"+str(i) for i in range(1,340) if i not in v_numbers_to_keep]
v_cols_to_drop[:6]

['V2', 'V5', 'V7', 'V9', 'V10', 'V12']

In [14]:
cols_to_drop = list(set(many_null_cols + one_value_cols  + v_cols_to_drop))
cols_to_drop = np.sort(cols_to_drop)
len(cols_to_drop)

256

In [15]:
train = train.drop(cols_to_drop, axis=1)
train.shape

(590540, 178)

# Limpieza de datos nulos

In [16]:
k = train.isna().sum()
k[k!=0]

card2           8933
card3           1565
card4           1577
card5           4259
card6           1571
               ...  
id_36         449555
id_37         449555
id_38         449555
DeviceType    449730
DeviceInfo    471874
Length: 158, dtype: int64

In [17]:
X = train.drop(columns = ["isFraud", "TransactionDT"], inplace = False)
y = train["isFraud"]
X.shape, y.shape

((590540, 176), (590540,))

In [18]:
X_n = X.loc[:,X.dtypes != object]
X_c = X.loc[:,X.dtypes == object]
X_n.shape, X_c.shape

((590540, 150), (590540, 26))

In [19]:
X_n.head()

,TransactionID,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,C1,...,V320,id_01,id_02,id_05,id_06,id_11,id_13,id_17,id_19,id_20
0,2987000,68.5,13926,NaN,150.0,142.0,315.0,87.0,19.0,1.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,29.0,2755,404.0,150.0,102.0,325.0,87.0,NaN,1.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,59.0,4663,490.0,150.0,166.0,330.0,87.0,287.0,1.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,50.0,18132,567.0,150.0,117.0,476.0,87.0,NaN,2.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,50.0,4497,514.0,150.0,102.0,420.0,87.0,NaN,1.0,...,0.0,0.0,70787.0,NaN,NaN,100.0,NaN,166.0,542.0,144.0


In [20]:
X_c.head()

,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,...,id_16,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,W,discover,credit,NaN,NaN,T,T,T,M2,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,W,mastercard,credit,gmail.com,NaN,NaN,NaN,NaN,M0,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,W,visa,debit,outlook.com,NaN,T,T,T,M0,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,W,mastercard,debit,yahoo.com,NaN,NaN,NaN,NaN,M0,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H,mastercard,credit,gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,...,NotFound,New,NotFound,samsung browser 6.2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


## Imputación var. numéricas X_train

In [21]:
from sklearn.impute import SimpleImputer
imp_median = SimpleImputer(missing_values=np.nan, strategy='median') # Imputación con la mediana

In [22]:
imp_median_test = SimpleImputer(missing_values=np.nan, strategy='median') # Imputación con la mediana
X_n_imp = pd.DataFrame(imp_median_test.fit_transform(X_n),
                   columns = X_n.columns)
X_n_imp = reduce_mem_usage(X_n_imp)
X_n_imp.head()

Mem. usage decreased to 179.09 Mb (73.5% reduction)


,TransactionID,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,C1,...,V320,id_01,id_02,id_05,id_06,id_11,id_13,id_17,id_19,id_20
0,2987000.0,68.5,13928.0,361.0,150.0,142.0,315.0,87.0,19.0,1.0,...,0.0,-5.0,125800.5,0.0,0.0,100.0,52.0,166.0,341.0,472.0
1,2987001.0,29.0,2756.0,404.0,150.0,102.0,325.0,87.0,8.0,1.0,...,0.0,-5.0,125800.5,0.0,0.0,100.0,52.0,166.0,341.0,472.0
2,2987002.0,59.0,4664.0,490.0,150.0,166.0,330.0,87.0,287.0,1.0,...,0.0,-5.0,125800.5,0.0,0.0,100.0,52.0,166.0,341.0,472.0
3,2987003.0,50.0,18128.0,567.0,150.0,117.0,476.0,87.0,8.0,2.0,...,0.0,-5.0,125800.5,0.0,0.0,100.0,52.0,166.0,341.0,472.0
4,2987004.0,50.0,4496.0,514.0,150.0,102.0,420.0,87.0,8.0,1.0,...,0.0,0.0,70787.0,0.0,0.0,100.0,52.0,166.0,542.0,144.0


In [23]:
X_n_imp.isna().sum().sum()

0

# Codificación e imputación de var. categóricas

In [24]:
X_c.nunique()

ProductCD           5
card4               4
card6               4
P_emaildomain      59
R_emaildomain      60
M1                  2
M2                  2
M3                  2
M4                  3
M5                  2
M6                  2
M7                  2
M8                  2
M9                  2
id_12               2
id_15               3
id_16               2
id_28               2
id_29               2
id_31             130
id_35               2
id_36               2
id_37               2
id_38               2
DeviceType          2
DeviceInfo       1786
dtype: int64

In [25]:
X_c["DeviceInfo"].value_counts()

Windows        47722
iOS Device     19782
MacOS          12573
Trident/7.0     7440
rv:11.0         1901
               ...  
LGMS345            1
verykool           1
XT1072             1
LG-H931            1
0PJA2              1
Name: DeviceInfo, Length: 1786, dtype: int64

DeviceInfo está mejor contenido en DeviceType, entonces, por simplicidad se eliminará la columna DeviceInfo.

In [26]:
X_c = X_c.drop(columns=["DeviceInfo"])
X_c.shape

(590540, 25)

In [27]:
assert "DeviceInfo" not in X_c.columns

In [28]:
 # Vamos a mapear los emails para mayor simplicidad.
emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum', 
          'scranton.edu': 'other', 'optonline.net': 'other', 'hotmail.co.uk': 'microsoft',
          'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo',
          'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft', 
          'aim.com': 'aol', 'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink',
          'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 'gmx.de': 'other',
          'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft', 
          'protonmail.com': 'other', 'hotmail.fr': 'microsoft', 'windstream.net': 'other', 
          'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo',
          'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other',
          'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft',
          'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink', 
          'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other', 
          'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo', 
          'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other', 
          'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 'cableone.net': 'other', 
          'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo', 'netzero.com': 'other', 
          'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other',
          'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple'}

us_emails = ['gmail', 'net', 'edu']

# https://www.kaggle.com/c/ieee-fraud-detection/discussion/100499#latest-579654
for c in ['P_emaildomain', 'R_emaildomain']:
    X_c[c + '_bin'] = X_c[c].map(emails)
    X_c[c + '_suffix'] = X_c[c].map(lambda x: str(x).split('.')[-1])
    X_c[c + '_suffix'] = X_c[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')


Eliminamos 'P_emaildomain', 'R_emaildomain', ya no son necesarias

In [29]:
X_c = X_c.drop(columns=['P_emaildomain', 'R_emaildomain'])
X_c.shape

(590540, 27)

In [30]:
X_c.head()

,ProductCD,card4,card6,M1,M2,M3,M4,M5,M6,M7,...,id_31,id_35,id_36,id_37,id_38,DeviceType,P_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_bin,R_emaildomain_suffix
0,W,discover,credit,T,T,T,M2,F,T,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan
1,W,mastercard,credit,NaN,NaN,NaN,M0,T,T,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,google,com,NaN,nan
2,W,visa,debit,T,T,T,M0,F,F,F,...,NaN,NaN,NaN,NaN,NaN,NaN,microsoft,com,NaN,nan
3,W,mastercard,debit,NaN,NaN,NaN,M0,T,F,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,yahoo,com,NaN,nan
4,H,mastercard,credit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,T,F,T,T,mobile,google,com,NaN,nan


## One-hot encoding

In [31]:
def to_onehot(x):
    values = np.unique(x)
    r = np.r_[[np.argwhere(i==values)[0][0] for i in x]]
    return np.eye(len(values))[r].astype(int)

def replace_column_with_onehot(d, col):
    assert sum(d[col].isna())==0, "column must have no NaN values"
    values = np.unique(d[col]
                      )
    k = to_onehot(d[col].values)
    r = pd.DataFrame(k, columns=["%s_%s"%(col, values[i]) for i in range(k.shape[1])], index=d.index).join(d)
    del(r[col])
    return r

In [32]:
dfs = []
for col in X_c.columns:
  dfs.append(replace_column_with_onehot(X_c[[col]].fillna("missing").copy(), col))

X_c_ohe = pd.concat(dfs, axis=1)
X_c_ohe = reduce_mem_usage(X_c_ohe)
X_c_ohe.head()

Mem. usage decreased to 157.48 Mb (85.9% reduction)


,ProductCD_C,ProductCD_H,ProductCD_R,ProductCD_S,ProductCD_W,card4_american express,card4_discover,card4_mastercard,card4_missing,card4_visa,...,R_emaildomain_bin_yahoo,R_emaildomain_suffix_com,R_emaildomain_suffix_de,R_emaildomain_suffix_es,R_emaildomain_suffix_fr,R_emaildomain_suffix_jp,R_emaildomain_suffix_mx,R_emaildomain_suffix_nan,R_emaildomain_suffix_uk,R_emaildomain_suffix_us
0,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [33]:
X_c_ohe.shape

(590540, 243)

In [34]:
del dfs
gc.collect()

0

# Dataset Limpio

In [35]:
final_X = pd.concat([X_n_imp, X_c_ohe], axis = 1)
final_X = reduce_mem_usage(final_X)
final_X.shape

Mem. usage decreased to 315.95 Mb (0.0% reduction)


(590540, 393)

In [36]:
y.shape

(590540,)

Particionamos los datos desde ya para que los modelos trabajen con los mismos datos.

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(final_X, y, test_size=0.30, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((413378, 393), (177162, 393), (413378,), (177162,))

In [38]:
X_train.head()

,TransactionID,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,C1,...,R_emaildomain_bin_yahoo,R_emaildomain_suffix_com,R_emaildomain_suffix_de,R_emaildomain_suffix_es,R_emaildomain_suffix_fr,R_emaildomain_suffix_jp,R_emaildomain_suffix_mx,R_emaildomain_suffix_nan,R_emaildomain_suffix_uk,R_emaildomain_suffix_us
366215,3353215.0,49.00000,17184.0,321.0,150.0,226.0,310.0,87.0,8.0,2.0,...,0,0,0,0,0,0,0,1,0,0
437716,3424716.0,20.53125,17152.0,273.0,185.0,197.0,299.0,87.0,8.0,2.0,...,0,1,0,0,0,0,0,0,0,0
578692,3565692.0,22.00000,16312.0,400.0,150.0,224.0,123.0,87.0,8.0,1.0,...,1,1,0,0,0,0,0,0,0,0
364816,3351816.0,86.00000,12600.0,111.0,150.0,166.0,494.0,87.0,4.0,1.0,...,0,0,0,0,0,0,0,1,0,0
223792,3210792.0,236.00000,7480.0,490.0,150.0,226.0,325.0,87.0,8.0,111.0,...,0,0,0,0,0,0,0,1,0,0


In [39]:
final_train = pd.concat((X_train, y_train), axis = 1)
final_train.head()

,TransactionID,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,C1,...,R_emaildomain_suffix_com,R_emaildomain_suffix_de,R_emaildomain_suffix_es,R_emaildomain_suffix_fr,R_emaildomain_suffix_jp,R_emaildomain_suffix_mx,R_emaildomain_suffix_nan,R_emaildomain_suffix_uk,R_emaildomain_suffix_us,isFraud
366215,3353215.0,49.00000,17184.0,321.0,150.0,226.0,310.0,87.0,8.0,2.0,...,0,0,0,0,0,0,1,0,0,0
437716,3424716.0,20.53125,17152.0,273.0,185.0,197.0,299.0,87.0,8.0,2.0,...,1,0,0,0,0,0,0,0,0,0
578692,3565692.0,22.00000,16312.0,400.0,150.0,224.0,123.0,87.0,8.0,1.0,...,1,0,0,0,0,0,0,0,0,0
364816,3351816.0,86.00000,12600.0,111.0,150.0,166.0,494.0,87.0,4.0,1.0,...,0,0,0,0,0,0,1,0,0,0
223792,3210792.0,236.00000,7480.0,490.0,150.0,226.0,325.0,87.0,8.0,111.0,...,0,0,0,0,0,0,1,0,0,0


In [40]:
final_test = pd.concat((X_test, y_test), axis = 1)
final_test.head()

,TransactionID,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,C1,...,R_emaildomain_suffix_com,R_emaildomain_suffix_de,R_emaildomain_suffix_es,R_emaildomain_suffix_fr,R_emaildomain_suffix_jp,R_emaildomain_suffix_mx,R_emaildomain_suffix_nan,R_emaildomain_suffix_uk,R_emaildomain_suffix_us,isFraud
470624,3457624.0,724.0000,7824.0,481.0,150.0,224.0,387.0,87.0,8.0,3.0,...,0,0,0,0,0,0,1,0,0,0
565820,3552820.0,108.5000,12544.0,321.0,150.0,226.0,476.0,87.0,0.0,2.0,...,0,0,0,0,0,0,1,0,0,0
284083,3271083.0,47.9375,9400.0,111.0,150.0,224.0,315.0,87.0,6.0,1.0,...,0,0,0,0,0,0,1,0,0,0
239689,3226689.0,100.6250,15888.0,545.0,185.0,138.0,299.0,87.0,8.0,2.0,...,1,0,0,0,0,0,0,0,0,0
281855,3268855.0,107.9375,15496.0,490.0,150.0,226.0,299.0,87.0,8.0,10.0,...,0,0,0,0,0,0,1,0,0,0


In [41]:
final_train.to_csv("train_clean.csv")
final_test.to_csv("test_clean.csv")


In [42]:
!zip data_clean.zip test_clean.csv train_clean.csv

  adding: test_clean.csv (deflated 92%)
  adding: train_clean.csv (deflated 92%)
